# Imports

In [1]:
from p7_01_functions import *

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline
plt.style.use('fivethirtyeight')
#sns.set()
#pd.options.display.max_columns = 100

# Read Master data

In [2]:
master_df = pd.read_csv('master_final_cleaned.csv')
master_df.head()
master_df.shape

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,client_credit_SK_DPD_DEF_min_count,client_credit_SK_DPD_DEF_min_mean,client_credit_SK_DPD_DEF_min_max,client_credit_SK_DPD_DEF_min_min,client_credit_SK_DPD_DEF_min_sum,client_credit_SK_DPD_DEF_sum_count,client_credit_SK_DPD_DEF_sum_mean,client_credit_SK_DPD_DEF_sum_max,client_credit_SK_DPD_DEF_sum_min,client_credit_SK_DPD_DEF_sum_sum
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,9461,-637,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,16765,-1188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,19046,-225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,19005,-3039,...,6.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,19932,-3038,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(307511, 1609)

### select a part of the entire data to detect important features due to memory issues

#### first shuffle the data and take a sample randomly

In [53]:
# master_df_cut0 = master_df.sample(10_000).copy() 
# master_df_cut = master_df_cut0.fillna(0).copy()
master_df_cut = master_df_cut.iloc[:,:50]

In [30]:
master_df_cut.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,client_bureau_balance_STATUS_X_count_count,client_bureau_balance_STATUS_X_count_mean,client_bureau_balance_STATUS_X_count_max,client_bureau_balance_STATUS_X_count_min,client_bureau_balance_STATUS_X_count_sum,previous_NAME_GOODS_CATEGORY_Animals_mean,previous_NAME_GOODS_CATEGORY_Animals_sum,previous_NAME_GOODS_CATEGORY_House Construction_mean,previous_NAME_GOODS_CATEGORY_House Construction_sum,previous_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal_mean
121570,240953,0,1,135000.0,193500.0,11106.0,193500.0,0.030755,15630,-2354,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
289266,435116,0,0,135000.0,495000.0,17779.5,495000.0,0.010276,12116,-4599,...,8.0,4.250000,11.0,0.0,34.0,0.0,0.0,0.0,0.0,0.0
829,100947,0,0,153000.0,380533.5,20772.0,328500.0,0.015221,9694,-1508,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59903,169446,0,1,157500.0,555273.0,18040.5,463500.0,0.019689,14770,-2189,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
281274,425852,0,2,202500.0,517500.0,33201.0,517500.0,0.018209,12953,-2061,...,6.0,12.166667,68.0,0.0,73.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# free memory
gc.enable()
del master_df, master_df_cut0
gc.collect()

0

# perform L1 regularization to select important features due to an important number of features

In [55]:
# instantiate Lasso
lasso_model = Lasso(alpha=1, normalize=True)


X = master_df_cut.drop(['TARGET','SK_ID_CURR'],axis=1).copy()
y = master_df_cut['TARGET'].copy()
#split data into train,test (80,20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15,random_state=7)

# Standardization
# scaler = StandardScaler()

# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# print(X_train_scaled.shape)

# fit lasso
lasso_model = lasso_model.fit(X_train, y_train)

#select important features
model = SelectFromModel(lasso_model, prefit=True)
X_regularised = model.transform(X_train)

print ("train score:",lasso_model.score(X_train,y_train))
print ("test score:",lasso_model.score(X_test,y_test))

train score: 0.0
test score: -5.145565582509448e-07


In [56]:
lasso_model.predict(X_test)

array([0.08152941, 0.08152941, 0.08152941, ..., 0.08152941, 0.08152941,
       0.08152941])

### show how much we can reduce the dimension

In [45]:
selected_features = pd.DataFrame(model.inverse_transform(X_regularised),
                                 index=X_train.index,
                                 columns=X_train.columns)



selected_columns = selected_features.columns[selected_features.var() != 0]

features =selected_features[selected_columns]

print("Before Lasso:",X_train.shape)
print("After Lasso:",features.shape)

ValueError: Found array with 0 feature(s) (shape=(8500, 0)) while a minimum of 1 is required.

In [ ]:
STOP

# instantiate xgboost model

In [ ]:
#instantiate XGB Regressor
xgbr = xgb.XGBRegressor(n_jobs=-1,objective='binary:logistic', eval_metric='auc')

#Hyper Parameter Optimization
params = {
   "learning_rate": [0.01, 0.05, 0.10],
    "max_depth": [4, 8, 12],
    "min_child_weight": [1,3,7],
    "gamma": [0.0,0.2,0.4],
    "colsample_bytree": [0.3,0.7],
    "n_estimators" : [100,300]
}

random_search = RandomizedSearchCV(xgbr,
                                   param_distributions=params,
                                   scoring='roc_auc',
                                   n_jobs=-1,cv=3,verbose=3)

# Build Xgboost Model

In [ ]:
# Prepare data to run rhe model
X = master_df_cut.drop(['TARGET','SK_ID_CURR'],axis=1).copy()
y = master_df_cut['TARGET'].copy()


#split data into train,test (80,20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15,random_state=7)

# Standardization
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

random_search.fit(X_train_scaled,y_train)
y_pred_test = random_search.predict(X_test_scaled)
y_pred_train = random_search.predict(X_train_scaled)

best_model = random_search.best_estimator_


print("train score:", r2_score(y_train,y_pred_train))
print("test score:", r2_score(y_test,y_pred_test))
important_features = print_importance_features(best_model, X_train.columns)

In [ ]:
random_search.best_estimator_